<a href="https://colab.research.google.com/github/ychoi-kr/llm-api-prog/blob/main/5_gemini/two_step_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai==0.8.1

In [ ]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key= userdata.get('GOOGLE_API_KEY'))

In [ ]:
import google.generativeai as genai
import google.ai.generativelanguage as glm

In [ ]:
# @title 2단계 함수 호출 구현하기
prod_database = {
    "갤럭시 S24": {"재고": 10, "가격": 1_700_000},
    "갤럭시 S23": {"재고": 5, "가격": 1_300_000},
    "갤럭시 S22": {"재고": 3, "가격": 1_100_000},
}

In [ ]:
from typing import Tuple
def is_product_available(product_name: str)-> Tuple[bool, str]:
    """특정 제품의 재고가 있는지 확인한다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        if prod_database[product_name]["재고"] > 0:
            return False, f"현재 {product_name} 재고가 있습니다."
    return False, f"현재 {product_name} 재고는 현재 없습니다."

def get_product_price(product_name: str)-> Tuple[bool, int]:
    """제품의 가격을 가져온다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        return True, prod_database[product_name]["가격"]
    return True, None

def place_order(product_name: str, address: str)-> Tuple[bool, str]:
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if is_product_available(product_name):
        prod_database[product_name]["재고"] -= 1
        return True, "주문 완료"
    else:
        return True, "재고 부족으로 주문 불가"


function_repoistory = {
    "is_product_available": is_product_available,
    "get_product_price": get_product_price,
    "place_order": place_order
}



In [ ]:
def correct_response(response):
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        for k, v in part.function_call.args.items():
            byte_v = bytes(v, "utf-8").decode("unicode_escape")
            corrected_v = bytes(byte_v, "latin1").decode("utf-8")
            part.function_call.args.update({k:  corrected_v})

In [ ]:
def make_fc_history(chat_session, part, answer):
    content = glm.Content(parts=[part], role="user")
    chat_session.history.append(content)

    response = glm.GenerateContentResponse({
        "candidates": [{"content": {"role": "model", "parts": [{"text": answer}]}}]
    })
    chat_session.history.append(response.candidates[0].content)
    return response

In [ ]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=function_repoistory.values()
)

chat_session = model.start_chat(history=[])
queries = ["갤럭시 S24 판매 중인가요?", "가격은 어떻게 되나요?", "서울시 종로구 종로1가 1번지로 배송부탁드립니다"]

for query in queries:
    print(f"\n사용자: {query}")
    #response = chat_session.send_message(query)
    print(f"history:\n{chat_session.history}")
    if query == "가격은 어떻게 되나요?":
        print(chat_session.send_message(query))
    else:
        response = chat_session.send_message(query)

    correct_response(response)
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        function_call =  part.function_call
        function_name = function_call.name
        function_args = {k: v for k, v in function_call.args.items()}
        print(function_repoistory[function_name](**function_args))
        is_required_3_4, function_result = function_repoistory[function_name](**function_args)
        part = glm.Part(
                function_response=glm.FunctionResponse(
                    name=function_name,
                    response={
                        "content": function_result,
                    },
                )
            )
        if is_required_3_4:
            response = chat_session.send_message(part)
        else:
            response = make_fc_history(chat_session, part, function_result)

        print(f"모델: {response.candidates[0].content.parts[0].text}")

print("="*100)
print(f"history:\n{chat_session.history}")



사용자: 갤럭시 S24 판매 중인가요?
history:
[]
(False, '현재 갤럭시 S24 재고가 있습니다.')
모델: 현재 갤럭시 S24 재고가 있습니다.

사용자: 가격은 어떻게 되나요?
history:
[parts {
  text: "갤럭시 S24 판매 중인가요?"
}
role: "user"
, parts {
  function_call {
    name: "is_product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "갤럭시 S24"
        }
      }
    }
  }
}
role: "model"
, parts {
  function_response {
    name: "is_product_available"
    response {
      fields {
        key: "content"
        value {
          string_value: "현재 갤럭시 S24 재고가 있습니다."
        }
      }
    }
  }
}
role: "user"
, parts {
  text: "현재 갤럭시 S24 재고가 있습니다."
}
role: "model"
]
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_product_price",
                  "args": {
                    